In [1]:
#! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py

In [2]:
import os

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [3]:
local_llm = 'llama3'

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document

In [22]:
urls = ["https://bkc.org/im-new/", 
       "https://bkc.org/",
       "https://bethelnews.s3.us-west-1.amazonaws.com/bethelnews_240512.pdf",
       "https://bkc.org/im-new/#hours"]

docs = [FireCrawlLoader(api_key="", url=url, mode="scrape").load() for url in urls]

docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=512, chunk_overlap=0)

In [6]:
doc_splits = text_splitter.split_documents(docs_list)

filtered_docs = []

for doc in doc_splits:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_docs.append(Document(page_content=doc.page_content, metadata=clean_metadata))


vectorstores = Chroma.from_documents(
    documents = filtered_docs,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(),
)

retriver = vectorstores.as_retriever()


In [81]:
#define your llm


llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
#llm = ChatOllama(model=local_llm, format="json", temperature=0)


In [82]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser


prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assesing relevance 
    of a retreived document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringend test. The goal is to filter out errouneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant><|end_header_id|>
    """,
    input_variables=["question", "document"],
)
retrieval_grader = prompt | llm | JsonOutputParser()
question = "who is the lead pastor and his name?"
docs = retriver.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))


{'score': 'yes'}


In [83]:
import getpass

In [84]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [88]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from transformers import pipeline


prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """, 
    input_variables=["question", "document"],
)


def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = prompt | llm | StrOutputParser()


# question = "what time is sunday service?"


docs = retriver.invoke(question)

generation = rag_chain.invoke({"context": docs, "question": question})

print(generation)

The lead pastor is 김한요 담임목사, whose English name is Rev. Bryan Hanyoh Kim.


In [89]:
#tavily API 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
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["TAVILY_API_KEY"] = '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'

web_search_tool = TavilySearchResults(k=3)

In [90]:
### Hallucination Grader

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
    answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no'
    score to indicate whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a single key 'score'
    and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents}
    \n ------- \n
    Here is the answer: {generation} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)


hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

In [91]:
### Answer Grader

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve the question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation. 
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],

)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"generation": generation, "question": question})



{'score': 'yes'}

In [92]:
from typing_extensions import TypedDict
from typing import List


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
       question: question
       generation: LLM generation
       web_search: whether to add search
       documents: list of documents
    """
    question: str
    generation: str
    web_search: str
    documents: List[str]

from langchain.schema import Document

def retrieve(state):
    """
    Retrieve documents from vector store
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    documents = retriver.invoke(question)
    return {"documents": documents, "question": question}


def grade_documents(state):
    """
    Determine whether the retrived documents are releavant to the question
    If not, we will detour to a web search (tavily)

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    
    #score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score["score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: Document Relevant---")
            filtered_docs.append(d)

        # Document not relevant
        else:
            print("---GRADE: Document Not Relevant---")
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def generate(state):
    """
    Generate response from LLM
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    generation = rag_chain.invoke({"context": docs, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def web_search(state):
    """
    Perform a web search using Tavily
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): Appended web results to document
    """
    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    docs = web_search_tool.invoke({"query": question})

    # not sure if this is correct
    web_results = "\n".join([d["content"] for d in docs])

    web_results = Document(page_content=web_results)

    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state
    
    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
    # All documents have been filtered check_relevance
    # we will re-generate a new query

        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT, INCLUDE WEB SEARCH---")
        return "web_search"
    else:
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score["score"]

    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        #check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"generation": generation, "question": question})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION IS NOT USEFUL---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"
    

from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

workflow.add_node("websearch", web_search)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)




In [93]:
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": END,
        "useful": END,
        "not useful": "websearch",
    },
)

In [98]:
#Compile
# question = "who is the lead pastor, what is his name?"

app = workflow.compile()

from pprint import pprint


inputs = {"question": "where is los angeles located?"}

for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running:{key}:")

print(value["generation"])


---RETRIEVE---
'Finished running:retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: Document Not Relevant---
---GRADE: Document Not Relevant---
---GRADE: Document Not Relevant---
---GRADE: Document Not Relevant---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT, INCLUDE WEB SEARCH---
'Finished running:grade_documents:'
---WEB SEARCH---
'Finished running:websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running:generate:'
Los Angeles is located in Southern California, United States. It is a major city in California, known for its Mediterranean climate, ethnic diversity, and the entertainment industry, particularly for being the home of Hollywood.
